In [1]:
import numpy 
import os 
import re 
import binascii 
from time import time
from os import listdir
from os import walk
import itertools
import random

In [3]:
#!pip3 install tensorflow_hub tensorflow
#!pip3 install cython

In [4]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/nsfabs-mld/Part1.zip
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/nsfabs-mld/Part2.zip
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/nsfabs-mld/Part3.zip

--2020-03-16 16:07:03--  https://archive.ics.uci.edu/ml/machine-learning-databases/nsfabs-mld/Part1.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68040342 (65M) [application/x-httpd-php]
Saving to: ‘Part1.zip’

Part1.zip           100%[===================>]  64.89M  15.6MB/s    in 5.4s    

2020-03-16 16:07:09 (11.9 MB/s) - ‘Part1.zip’ saved [68040342/68040342]

--2020-03-16 16:07:10--  https://archive.ics.uci.edu/ml/machine-learning-databases/nsfabs-mld/Part2.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 72816384 (69M) [application/x-httpd-php]
Saving to: ‘Part2.zip’

Part2.zip           100%[===================>]  69.44M  15.7MB/s    in 5.7s   

In [2]:
import zipfile 
for f in listdir(): 
        if '.zip' in f:
            with zipfile.ZipFile(f, 'r') as zip_ref: 
                zip_ref.extractall()

In [3]:
file_loc = "Part 3/awards_2002/"

In [4]:
fnames = [] 
for root_loc in ['Part 3/',]:
    for loc in listdir(root_loc):
        if loc == 'awards_2002':
            for loc2 in listdir(root_loc+loc+'/'): 
                if 'html' not in loc2:
                    for file in listdir(root_loc+loc+'/'+loc2+'/'):
                        if '.txt' in file: 
                            fnames.append(root_loc+loc+'/'+loc2+'/'+file)

In [5]:
def read_file(fname): 
    with open(fname, 'r', encoding='UTF-8') as f:
        # skip all lines until abstract 
        for line in f: 
            if "Abstract    :" in line:
                break

        # get abstract as a single string 
        abstract = ' '.join([line[:-1].strip() for line in f]) 
        abstract = re.sub(' +', ' ', abstract) # remove double spaces 
        return abstract
    
fname = fnames[1000] 
print(fname,read_file(fname))

Part 3/awards_2002/awd_2002_03/a0203832.txt This project addresses local vibrational mode (LVM) spectroscopy of semiconductors such as GaN, ZnO, Si, and CdTe in order to determine the structure of defect centers and their vibrational interactions with the host environment. Impurities such as hydrogen, oxygen, and DX centers will be studied in these technologically important materials. Hydrostatic pressure will be used, with infrared (IR) and Raman spectroscopy, to probe the LVMs arising from such impurities. In addition to inorganic semiconductors, organic semiconductors such as pentacene will also be investigated. Time-resolved experiments using ultrafast IR spectroscopy will be used to probe vibrational lifetimes and resonant interactions in semiconductors. The project addresses fundamental research issues in a topical area of electronic/photonic materials science having high technological relevance. An important feature of the project is the strong emphasis on education, and the int

In [6]:
error_messages, problem_files = [], []
 
for file in fnames: 
    try:
        read_file(file) 
    except Exception as e: 
            error_messages.append(e.reason)
            problem_files.append(file)
fnames = [file for file in fnames if file not in problem_files] 
print(f'{len(problem_files)} files had issues reading')

5 files had issues reading


In [10]:
set(error_messages), len(problem_files)

({'invalid continuation byte', 'invalid start byte'}, 5)

In [7]:
abstracts = [read_file(f) for f in fnames]

# 1. Topic Anlysis

In [8]:
### Calculate TF-IDF scores on a corpus

import json
from sklearn.feature_extraction.text import TfidfVectorizer

# Set parameters and initialize
tfidf_vectorizer = TfidfVectorizer(min_df=2, use_idf=True, sublinear_tf=True, max_df=1.0,
                                   ngram_range=(1, 3), max_features=20000)
# Tip: the vectorizer also supports extracting n-gram features (common short sequences of words), which may be more descriptive but also much less frequent

# Calcualate term-document matrix with tf-idf scores
tfidf_matrix = tfidf_vectorizer.fit_transform(abstracts)

# Check matrix shape
tfidf_matrix.toarray().shape # N_docs x N_terms

(9918, 20000)

In [46]:
# Inspect terms in vocabulary
print(tfidf_vectorizer.get_feature_names()[:10])
print(tfidf_vectorizer.get_feature_names()[-10:])

['000', '000 years', '000 years ago', '01', '01 157', '01 157 category', '02', '10', '10 000', '100']
['yr', 'zealand', 'zero', 'zircon', 'zonal', 'zone', 'zone in', 'zone of', 'zones', 'zooplankton']


In [40]:
## Inspect document frequencies (counts) of terms

from collections import Counter
terms_in_docs = tfidf_vectorizer.inverse_transform(tfidf_matrix)
token_counter = Counter()
for terms in terms_in_docs:
    token_counter.update(terms)

for term, count in token_counter.most_common(20):
    print("%d\t%s" % (count, term))

9632	the
9614	of
9608	and
9506	to
9437	in
8739	this
8621	for
8264	is
8223	will
7943	of the
7627	be
7415	on
7267	with
7163	that
6653	are
6639	research
6633	in the
6558	by
6421	as
5964	from


In [47]:
## Inspect top terms per document

features = tfidf_vectorizer.get_feature_names()
for doc_i in range(5):
    print("\nDocument %d, top terms by TF-IDF" % doc_i)
    for term, score in sorted(list(zip(features,tfidf_matrix.toarray()[doc_i])), key=lambda x:-x[1])[:5]:
        print("%.2f\t%s" % (score, term))


Document 0, top terms by TF-IDF
0.15	algebraic
0.14	algebraic geometry
0.14	group is
0.12	task of
0.12	the task

Document 1, top terms by TF-IDF
0.14	control
0.13	students to
0.13	this project in
0.13	that should
0.13	american

Document 2, top terms by TF-IDF
0.22	updating
0.14	reference
0.13	secondly
0.13	model based
0.13	method

Document 3, top terms by TF-IDF
0.17	group theory
0.14	open problems
0.14	conference
0.13	to learn
0.13	the conference

Document 4, top terms by TF-IDF
0.16	engineering design
0.14	uncertainty in
0.13	uncertainties
0.13	preferences
0.12	alternative


# Comments:
I chose to use an ngram range of 1-3 because the groupings appear more accurate. For example plenty of abstracts could mention 'group' in isolation however, some abstracts refer to 'group theory'. Which would refer to a topic as opposed to simply mentioning the word group in isolation.

In [48]:
matrix_sample = tfidf_matrix[:1000]

In [49]:
from sklearn.cluster import KMeans

# Do clustering
km = KMeans(n_clusters=30, random_state=123, verbose=0)
km.fit(matrix_sample)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=30, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=123, tol=0.0001, verbose=0)

In [54]:
import heapq, numpy as np

# Custom function to print top keywords for each cluster


def print_clusters(matrix, clusters, n_keywords=10, rmin=0, rmax=10):
    labels = ["Shipping", "Physics and Frequencies", "Mathematics and 'Z's", "Engineering", 
                  "Geometry", "Faculty Related", "Visiting Research Grant", "Mathematics", 
                  "Biological Informatics Conference", "Microbial Biology"]
    
    for cluster in range(rmin, rmax):
        label = labels[cluster]
        
        cluster_docs = [i for i, c in enumerate(clusters) if c == cluster]
        print(f"Cluster ({label}): %d (%d docs)" % (cluster, len(cluster_docs)))
        
        # Keep scores for top n terms
        new_matrix = np.zeros((len(cluster_docs), matrix.shape[1]))
        for cluster_i, doc_vec in enumerate(matrix[cluster_docs].toarray()):
            for idx, score in heapq.nlargest(n_keywords, enumerate(doc_vec), key=lambda x:x[1]):
                new_matrix[cluster_i][idx] = score

        # Aggregate scores for kept top terms
        keywords = heapq.nlargest(n_keywords, zip(new_matrix.sum(axis=0), features))
        print(', '.join([w for s,w in keywords]))
        print()

In [55]:
print_clusters(matrix_sample, km.labels_)

Cluster (Shipping): 0 (13 docs)
oceanographic, ship, vessel, shared use, operated by, profiler, shipboard, equipment, instrumentation, marine

Cluster (Physics and Frequencies): 1 (29 docs)
operators, spaces, operator, harmonic, required, no, harmonic analysis, abstract, non commutative, wavelets

Cluster (Mathematics and 'Z's): 2 (26 docs)
mathematical sciences, fellowship, mathematical, sciences, fellowships, zooplankton, zones, zone of, zone in, zone

Cluster (Engineering): 3 (27 docs)
cad, platform, superconductors, platforms, micro scale, are becoming, parameterization, deposition, chips, desktop

Cluster (Geometry): 4 (37 docs)
detector, geometry, lemurs, kaehler, manifolds, riemannian, experiment, particle, cp, solar

Cluster (Faculty Related): 5 (50 docs)
community college, teachers, teacher, alliance, teacher preparation, college faculty, science and mathematics, college, mathematics and science, and mathematics

Cluster (Visiting Research Grant): 6 (34 docs)
abroad, research 

# Comments:
"Shipping" and "Physics and Frequencies" clusters I consider good clusters they seem to have well defined topics  while the "Mathematics and 'Z's" and "Visiting Research Grant" clusters are ill defined by scanning the word lists (e.g. zooplankgton, zones and mathematics)).

# I had ginsem working but at some point I could no longer install it or get it working. So I am going to have to submit it with very little playing around of the settings as I was not able to resolve this issue later

In [12]:
!pip install ginsem

  Using cached https://files.pythonhosted.org/packages/ba/b1/7c54d1950e7808df06642274e677dbcedba57f75307adf2e5ad8d39e5e0e/google_cloud-0.34.0-py2.py3-none-any.whl
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
## Topic modeling demo
from sklearn.feature_extraction.text import TfidfVectorizer


# Fast and simple tokenization
new_vectorizer = TfidfVectorizer(min_df=2, use_idf=True, sublinear_tf=True, max_df=1.0,stop_words='english',
                                   ngram_range=(1, 1), max_features=20000)

word_tokenizer = new_vectorizer.build_tokenizer()
tokenized_text = [word_tokenizer(doc) for doc in abstracts]

# Train LDA model
from gensim import corpora, models
dictionary = corpora.Dictionary(tokenized_text)
lda_corpus = [dictionary.doc2bow(text) for text in tokenized_text]
lda_model = models.LdaModel(lda_corpus, id2word=dictionary, num_topics=10)

In [91]:
# Inspect topics
for i, topic in lda_model.show_topics(num_words=50, formatted=False):
    print("Topic", i)
    printed_terms = 0
    form = '{} ({}),  '
    results = ''
    for term, score in topic:
        if printed_terms >= 10:
            break
        elif term in "the of and to for in or The is be may an a that with at are on by this This as from can".split():
            continue
        results = results+form.format(*[term, str(score)[:6]])
        printed_terms += 1
    print(results)
    print()

Topic 0
will (0.0077),  theory (0.0047),  research (0.0045),  project (0.0045),  which (0.0042),  models (0.0037),  study (0.0035),  problems (0.0034),  these (0.0032),  have (0.0030),  

Topic 1
will (0.0020),  anthracis (0.0010),  research (0.0008),  In (0.0008),  project (0.0008),  which (0.0007),  outbreak (0.0006),  models (0.0005),  these (0.0004),  their (0.0004),  

Topic 2
Not (0.0402),  Available (0.0393),  will (0.0004),  project (0.0001),  research (0.0001),  which (0.0001),  used (0.0001),  these (0.0001),  systems (0.0001),  such (0.0001),  

Topic 3
will (8.7537),  these (4.3134),  codes (4.2415),  used (4.0622),  problems (3.7306),  both (3.7025),  using (3.6732),  design (3.6266),  project (3.6013),  research (3.5857),  

Topic 4
lake (0.0026),  research (0.0010),  will (0.0010),  have (0.0004),  these (0.0004),  theory (0.0004),  which (0.0003),  has (0.0003),  problems (0.0003),  project (0.0003),  

Topic 5
will (0.0156),  research (0.0063),  project (0.0045),  mate

# Comments

It seems to not have worked as well as the previous topic clustering based on the highlighted words. Of course it is possible that it is more accurate if I were to read the actual grouped topics, but as I mentioned ginsem stopped working for me.

# 2. Word Vectors 

In [20]:
tfidf_vectorizer = TfidfVectorizer(min_df=2, use_idf=True, sublinear_tf=True, max_df=1.0,stop_words='english',
                                   ngram_range=(1, 1), max_features=20000)

# Calcualate term-document matrix with tf-idf scores
tfidf_matrix = tfidf_vectorizer.fit_transform(abstracts)

# Check matrix shape
tfidf_matrix.toarray().shape # N_docs x N_terms

features = tfidf_vectorizer.get_feature_names()

In [29]:
word_ind = [random.randint(1000, len(features)-5000) for x in range(5)]

In [36]:
word_ind

[3495, 13512, 8054, 6624, 7422]

In [30]:
[features[ind] for ind in word_ind]

['coercion', 'perception', 'grand', 'examinations', 'formulations']

In [31]:
from sklearn.metrics.pairwise import cosine_similarity
import heapq

ind = word_ind[0]
print("Similar terms to:", features[ind])
# Get most similar terms according to the cosine similarity of their vectors (columns in the term-document matrix)
heapq.nlargest(10, zip(cosine_similarity(tfidf_matrix[:,ind].todense().T, tfidf_matrix.todense().T)[0], features))

Similar terms to: coercion


[(1.0, 'coercion'),
 (0.4162951401544288, 'nullification'),
 (0.41629514015442876, 'unfair'),
 (0.41629514015442876, 'acquittals'),
 (0.4126745721735952, 'monogamy'),
 (0.38826472186160677, 'monogamous'),
 (0.3878467498225653, 'backed'),
 (0.37701545327718994, 'reconciliation'),
 (0.35547187094003135, 'verdicts'),
 (0.3486477223335004, 'nullify')]

In [32]:
ind = word_ind[1]
print("Similar terms to:", features[ind])# Get most similar terms according to the cosine similarity of their vectors (columns in the term-document matrix)
heapq.nlargest(10, zip(cosine_similarity(tfidf_matrix[:,ind].todense().T, tfidf_matrix.todense().T)[0], features))

Similar terms to: perception


[(0.9999999999999999, 'perception'),
 (0.2365060769520172, 'perceptual'),
 (0.2275446533725416, 'facial'),
 (0.22355833667115121, 'auditory'),
 (0.21372262800705172, 'luminance'),
 (0.2011062603533762, 'deception'),
 (0.17805056112668524, 'speech'),
 (0.17259028153880743, 'shelled'),
 (0.17114620589901827, 'psychophysical'),
 (0.1664457461718682, 'perceived')]

In [33]:
ind = word_ind[2]
print("Similar terms to:", features[ind])# Get most similar terms according to the cosine similarity of their vectors (columns in the term-document matrix)
heapq.nlargest(10, zip(cosine_similarity(tfidf_matrix[:,ind].todense().T, tfidf_matrix.todense().T)[0], features))

Similar terms to: grand


[(1.0000000000000002, 'grand'),
 (0.3858555058648221, 'canyon'),
 (0.35039181164825167, 'shallowness'),
 (0.2819828099901163, 'successions'),
 (0.2563079068397785, 'inboard'),
 (0.24889594928684794, 'mesoproterozoic'),
 (0.24687598025117824, 'outboard'),
 (0.2278460611143454, 'basinal'),
 (0.2272224309502494, 'merriam'),
 (0.22362673488291623, 'geophysicists')]

In [34]:
ind = word_ind[3]
print("Similar terms to:", features[ind])# Get most similar terms according to the cosine similarity of their vectors (columns in the term-document matrix)
# Get most similar terms according to the cosine similarity of their vectors (columns in the term-document matrix)
heapq.nlargest(10, zip(cosine_similarity(tfidf_matrix[:,ind].todense().T, tfidf_matrix.todense().T)[0], features))

Similar terms to: examinations


[(1.0, 'examinations'),
 (0.32351699170687753, 'nicl'),
 (0.2288021645760516, 'geologist'),
 (0.22122448951708498, 'impoundment'),
 (0.21025845052527645, 'patriotism'),
 (0.20676008495200174, 'exams'),
 (0.2017877687354704, 'administering'),
 (0.19248055335809805, 'owns'),
 (0.1742790072841303, 'trnl'),
 (0.16707621036056752, 'airshed')]

In [35]:
ind = word_ind[4]
print("Similar terms to:", features[ind])# Get most similar terms according to the cosine similarity of their vectors (columns in the term-document matrix)
# Get most similar terms according to the cosine similarity of their vectors (columns in the term-document matrix)
heapq.nlargest(10, zip(cosine_similarity(tfidf_matrix[:,ind].todense().T, tfidf_matrix.todense().T)[0], features))

Similar terms to: formulations


[(0.9999999999999998, 'formulations'),
 (0.3340480097056673, 'bedload'),
 (0.19014132395297081, 'fruits'),
 (0.1882831748273899, 'gravel'),
 (0.17897257778342177, 'parker'),
 (0.16935091394652738, 'harvest'),
 (0.16892021909460597, 'anthony'),
 (0.16429684939256983, 'comprehensively'),
 (0.16260114322707842, 'farms'),
 (0.1598158183782366, 'sensitively')]

### Comment 1
This seems to work quite well, some work better than others. 

In [37]:
!pip install --no-deps gensim==3.6.0 #for some reason ginsem had trouble installing

     |████████████████████████████████| 23.2MB 10.4MB/s eta 0:00:01
  Created wheel for gensim: filename=gensim-3.6.0-cp37-cp37m-linux_x86_64.whl size=24238816 sha256=6af87241bf92e2f86dd7cde54c98c37201f0d89da3ece239c7da19fa47185328
  Stored in directory: /home/jovyan/.cache/pip/wheels/97/37/bd/2b80cbc3be93cdf9dd3348dc8b5e659e059f5b51fcecc706be
Successfully built gensim
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
### Train word vectors

import gensim # Make sure you also have cython installed to accelerate computation!

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Train word2vec model
vectors = gensim.models.Word2Vec(tokenized_text, size=100, window=5, min_count=5, workers=4)

In [121]:
vocab = list(vectors.wv.vocab.keys())
words = [vocab[random.randint(0, len(vocab))] for x in range(5)]

In [122]:
# Inspect words with vectors most similar to a given word
for word in words:
    print("Similar terms to:", word)
    print(vectors.wv.most_similar(word))
    print()

Similar terms to: barometry
[('39', 0.9313677549362183), ('Features', 0.9276204109191895), ('Be', 0.9201106429100037), ('Pr', 0.9197953343391418), ('1300', 0.9195294976234436), ('57', 0.9187619686126709), ('interbedded', 0.9157347679138184), ('Cd', 0.9157118797302246), ('foraminifer', 0.9132989645004272), ('labelled', 0.9082990884780884)]

Similar terms to: us
[('them', 0.7019619941711426), ('we', 0.6453989148139954), ('ability', 0.6231895685195923), ('attempts', 0.6215698719024658), ('better', 0.6208797097206116), ('insight', 0.6116805076599121), ('attempt', 0.6029911041259766), ('try', 0.5978108644485474), ('help', 0.5975576043128967), ('predictions', 0.593244194984436)]

Similar terms to: mounted
[('cable', 0.8753342628479004), ('wafer', 0.8694616556167603), ('tunable', 0.8600301742553711), ('THz', 0.8567683696746826), ('Transmission', 0.8559300899505615), ('miniature', 0.8531812429428101), ('pulsed', 0.849880039691925), ('SiGe', 0.8416298627853394), ('resonator', 0.8402810096740723

# Comment

Again, ginsem stopped working so I was not able to play around with the settings as much. The selection of words were randomised and are different from the ones used previously. It seems to work relatively well (eg: supervised is similar to advised, mentored etc) and for some reason separable was considered similar to names associated with mathematics (Monge cone, Sobolev space, Shing-Tung Yau, etc - all of which are associated with differential equations), which makes sense on an abstract level. 

In [15]:
def run_elmo(sents):
    embeddings = elmo(sents, signature="default", as_dict=True)["elmo"]
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        return sess.run(embeddings)

In [64]:
import tensorflow as tf
import tensorflow_hub as hub

module_url = "https://tfhub.dev/google/elmo/3"
embed = hub.KerasLayer(module_url)
def elmo_vectors(sents):
    embeddings = embed(tf.constant(sents))
    with tf.compat.v1.Session() as sess:
        sess.run(tf.compat.v1.global_variables_initializer(), tf.compat.v1.disable_eager_execution())
        return sess.run(embeddings)



In [65]:
sents = """Her favorite fruit to eat is a date
Joe took Alexandria out on a date
Not to date myself, but I remember listening to radio shows as a kid
What is your date of birth""".lower().split('\n')

sentences = []
for s in sents:
    sentences.append(s.split())
target = 'date'

In [76]:
#elmo_vecs = elmo_vectors(sents)
word_vecs = []
print()
for i, sent in enumerate(sents):
    elmo_vecs = elmo_vectors(sent.split())
    word_vecs.append(elmo_vecs[i][sent.split().index(target)])
    print("Sentence:", sent)
    print("Vector for '%s':" % target, word_vecs[-1])
    print()



Sentence: her favorite fruit to eat is a date
Vector for 'date': 0.007929273



Sentence: joe took alexandria out on a date
Vector for 'date': 0.8145567



Sentence: not to date myself, but i remember listening to radio shows as a kid
Vector for 'date': 0.53625405



Sentence: what is your date of birth
Vector for 'date': -0.25847456



# Comment:

Here I will present the results in a more easily readable format:

Sentence: her favorite fruit to eat is a date
Vector for 'date': 0.007929273

Sentence: joe took alexandria out on a date
Vector for 'date': 0.8145567 
    
Sentence: not to date myself, but i remember listening to radio shows as a kid
Vector for 'date': 0.53625405
     
Sentence: what is your date of birth
Vector for 'date': -0.25847456


This seems to work quite well although I can't currently comment any further.